# Data Exploration & Feature Engineering

## Overview

In this section we will perform an overview on wine dataset. It's hard to know what to do if you don't know what you're working with, so let's load our dataset and take a peek. We already know that there is only one target column: `quality`. We will discover the dataset size and number of features. We will also look the first rows to figure out all the column types.

In [8]:
import numpy as np
import pandas as pd
from time import time

%matplotlib inline

# Read wine dataset
data = pd.read_csv("data.csv", delimiter=";")

# Print the first rows
display(data.head())

# Calculate number of wines in dataset
n_wines = data.shape[0]

# Calculate number of features
n_features = data.shape[1] - 1

print("Total number of wines: {}".format(n_wines))
print("Number of features: {}".format(n_features))


,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,White,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,White,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,White,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,White,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,White,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


Total number of wines: 6497
Number of features: 12


As we can see, the dataset size is 6497 and it has 12 fetures. By looking the firt rows of the dataset, we have the following column types:

  - `type`: Categorical (nominal)
  - `fixed acidity`: Numeric (continuous)
  - `volatile acidity`: Numeric (continuous)
  - `citric acid`: Numeric (continuous)
  - `residual sugar`: Numeric (continuous)
  - `chlorides`: Numeric (continuous)
  - `free sulfur dioxide`: Numeric (continuous)
  - `total sulfur dioxide`: Numeric (continuous)
  - `density`: Numeric (continuous)
  - `pH`: Numeric (continuous)
  - `sulphates`: Numeric (continuous)
  - `alcohol`: Numeric (continuous)
  - `quality`: Numeric (ordinal)



# Preparing data

In this section, we will prepare the data for modeling, training and testing.

## Feature columns vs target column

It is often the case that the data contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

The code cell below will separate the wine data into feature and target columns to see if any features are non-numeric.

In [ ]:
# Extract feature columns
feature_cols = list(data.columns[:-1])

# Extract target column 'quality'
target_col = data.columns[-1]

# Show the list of columns
print("\nTarget column: {}".format(target_col))

print("\nFeature types:")
print(data[feature_cols].dtypes)

print("\nTarget type:")
print(data[target_col].dtypes)

print("\nClasses from `type` feature:")
print(data['type'].unique())

## Preprocess features

As we can see, there two non-numeric features that need to be converted: `type` and `alcohol`.

The `type` feature only has two classes (`['White' 'Red']`), so it can be reasonably converted into 1/0 (binary) values.

The `alcohol` feature is actually a numeric feature, but some rows are wrong non-numeric values, for example `'110.666.666.666.667'`, `'956.666.666.666.667'`, `'923.333.333.333.333'` and so on. Therefore, we will parse the values of this feature into numeric values and replace the wrong values with the mean of all of the values for the feature ([reference](https://machinelearningmastery.com/handle-missing-data-python/)).

The code cell below performs the preprocessing routine discussed.
 


In [ ]:

def preprocess_features(X):
    ''' Preprocesses the data and converts non-numeric binary variables into
        binary (0/1) variables.'''

    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If type, replace all 'White'/'Red' values with 1/0
        if col == 'type' and col_data.dtype == object:
            col_data = col_data.replace(['White', 'Red'], [1, 0])
        
        # If alcohol, parse it values to numberic and coerce the convertion errors
        if col == 'alcohol' and col_data.dtype == object:
            col_data = pd.to_numeric(col_data, errors='coerce')

        # Collect the revised columns
        output = output.join(col_data)

    # handle NaN values by inputting the mean
    output.fillna(output.mean(), inplace=True)
    
    return output

data = preprocess_features(data)

## Outliers and Feature scaling

We've successfully cleaned our data and converted it into a form which is easily consumable by machine learning algorithms. However, at this point we should consider whether or not some method of data normalization / scaling or outlier removal will be beneficial for our algorithm.

Feature normalization / scaling allows for all features to contribute equally (or more aptly, it allows for features to contribute relative to their importance rather than their scale).

Outliers are extreme values that deviate from other observations on data , they may indicate a variability in a measurement, experimental errors or a novelty. In other words, an outlier is an observation that diverges from an overall pattern on a sample. Some machine learning are sensitive to outliers.

The code cell below plots the boxplot for all features in dataset.

In [ ]:
import seaborn as sns
sns.boxplot(data=data[feature_cols], orient="h", palette="Set2")

As we can see, the features contain many (extreme) outliers. In the code cell bellow we will create a function to  perform outlier removal based on boxplot method and IQR factor (default == 1.5) ([reference](https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba)). In a later section, we will vary this factor to analyse the best threshold to remove the outliers.

In [ ]:
def remove_outliers(data_orig, feature_cols, target_col, factor=1.5, verbose=False):
    data = data_orig.copy()
    features = feature_cols
    Q1 = data[features].quantile(0.25)
    Q3 = data[features].quantile(0.75)
    IQR = Q3 - Q1    
    is_outlier = (data[features] < (Q1 - factor * IQR)) |(data[features] > (Q3 + factor * IQR))
    data = data[~(is_outlier).any(axis=1)]
    if verbose:
        print("Number of outliers for IQR factor {}: {}".format(factor, data_orig.shape[0] - data.shape[0]))
    return data[feature_cols], data[target_col]

test1 = remove_outliers(data, feature_cols, target_col, verbose=True)
test1 = remove_outliers(data, feature_cols, target_col, factor=3.5, verbose=True)

In the boxplot, the range of values varies widely between the features. We will create a function to normalize features using min-max or standard methods so that each feature contributes proportionately in the same scale in the models.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def scale(X, scaler_type='minmax'):
    X = X.copy() # TODO
    scaler = MinMaxScaler()
    if scaler_type == 'minmax':
        scaler = MinMaxScaler()
    elif scaler_type == 'standard':
        scaler = StandardScaler()

    features = list(X.select_dtypes(include=['float64']).columns)
    X[features] = scaler.fit_transform(X[features])
    return X


## Data balancing

A very frequent problem in machine learning models is imbalanced dataset, because an imbalanced dataset may lead to inflated performance estimates ([reference](https://stats.stackexchange.com/questions/28029/training-a-decision-tree-against-unbalanced-data)). Therefore, We will analyse if the dataset response variable is balalanced or not.

In [ ]:
print("Bar plot for the counts of observations in each quality class.")
sns.catplot("quality", data=data, hue='quality', kind="count")
print(data[target_col].value_counts())

As we can see, the response variable `quality` is very unbalanced. The code cell bellow will balance the minority classes by up-sample them. Up-sampling is the process of randomly duplicating observations from the minority class in order to reinforce its signal ([reference](https://elitedatascience.com/imbalanced-classes)).

In [ ]:
from sklearn.utils import resample

def upsample(data, target_col, majority_class):
    '''
    Upsample minority class based on majority_class size of a give target column
    '''
    target_classes = data[target_col].unique()
    majoritary_df = data[data[target_col] == majority_class]
    majoritary_size = majoritary_df.shape[0]
    frames = [majoritary_df]

    # combine majority class data frame with upsampled minority ones
    for target_class in target_classes:
        if target_class == majority_class:
            continue
        df = data[data[target_col] == target_class]
        df_upsampled = resample(df, replace=True, n_samples=majoritary_size, random_state=123)
        frames.append(df_upsampled)
    data_balanced = pd.concat(frames)
    return data_balanced

target_majority_class = 6
data_balanced = upsample(data, target_col, target_majority_class)

print("Bar plot for the counts in each quality class after data balance.")
sns.catplot("quality", data=data_balanced, hue='quality', kind="count")
print(data_balanced[target_col].value_counts())

# Model Evaluation
We will now import the three supervised learning models to perform the **Cross Validation** for each one using the balanced training/testing datasets.

**Cross Validation** is used to assess the predictive performance score of the models and and to judge how they perform outside the sample to a new data set also known as test data. The motivation to use cross validation techniques is that when we fit a model, we are fitting it to a training dataset. Without cross validation we only have information on how does our model perform to our in-sample data. Ideally we would like to see how does the model perform when we have a new data in terms of **accuracy** of its predictions. There two types of cross validation you can perform: leave one out and **k fold** ([reference](https://www.researchgate.net/post/What_is_the_purpose_of_performing_cross-validation)). We will use the k_fold type.

To find the best model, we will validate the **accuracy score** for each classifier for four different **training set sizes** (25%, 50%, 75% and 100%) and two different **normalization methods** (min-max, standard). For didactic purposes, we will also remove the outliers from the training set based on different **IQR factors** (1.5, 2.0, 2.5, 3.0) to train each classifier.

In [ ]:
from sklearn.model_selection import cross_val_score

# http://scikit-learn.org/stable/modules/multiclass.html
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import warnings
warnings.filterwarnings('ignore')

def cross_validation(classifier, X, y, cv=10, size_proportions=[0.25, 0.50, 0.75, 1.0]):
    '''
    Execute the 'cross_val_score' function for a classifier, each 
    training set size proportion (25%, 50%, 75%, 100%), and each
    normalization type (min-max, standard)
    '''
    scores = []
    total_size = X.shape[0]
    sizes = (map(lambda p: int(total_size * p), size_proportions))
    X_minmax = scale(X, scaler_type='minmax')
    X_standard = scale(X, scaler_type='standard')

    for size in sizes:
        scores1 = cross_val_score(classifier, X[:size], y[:size], cv=cv)
        scores2 = cross_val_score(classifier, X_minmax[:size], y[:size], cv=cv)
        scores3 = cross_val_score(classifier, X_standard[:size], y[:size], cv=cv)
        scores.append([size, np.mean(scores1), np.mean(scores2), np.mean(scores3)])
    result = pd.DataFrame(scores, columns=['size(X)', 'X', 'minmax(X)', 'standard(X)'])
    return result

# Initialize the models
clf_A = DecisionTreeClassifier(random_state=0)
clf_B = RandomForestClassifier(n_estimators=100)
clf_C = LinearDiscriminantAnalysis()  # Variables must not be collinear

classifiers = [clf_A, clf_B, clf_C]
iqr_factors = [1.5, 2.0, 2.5, 3.0]
size_proportions=[0.25, 0.50, 0.75, 1.0]
cv = ShuffleSplit(n_splits=7, test_size=0.2, random_state=0)

# Execute 'cross_validation' for each classifier, normalization type, training set size and iqr_factor
for clf in classifiers:
    print("\n[{}]\n".format(clf.__class__.__name__))

    print('Scores considering outliers')
    X_all, y_all = data_balanced[feature_cols], data_balanced[target_col]
    scores = cross_validation(clf, X_all, y_all, cv=cv)
    display(scores)

    for iqr_factor in iqr_factors:
        print('Scores considering no outliers, IQR factor', iqr_factor)
        X_all, y_all = remove_outliers(data_balanced, feature_cols, target_col, factor=iqr_factor)
        scores = cross_validation(clf, X_all, y_all, cv=k_fold)
        display(scores)




# TODO sklearn.ensemble.GradientBoostingClassifier
# http://scikit-learn.org/stable/modules/model_evaluation.html
# https://elitedatascience.com/imbalanced-classes
# https://stackoverflow.com/questions/26210471/scikit-learn-gridsearch-giving-valueerror-multiclass-format-is-not-supported
#     print(y.unique())
#     y = label_binarize(y, classes=y.unique())
#     print(y)

## Learning Curve

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from plot_learning_curve import plot_learning_curve
print(plot_learning_curve)

X_all, y_all = data_balanced[feature_cols], data_balanced[target_col]
title = "Learning Curves (Decision Tree)"
# Cross validation with 7 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=7, test_size=0.2, random_state=0)
estimator = DecisionTreeClassifier(random_state=0)
plot_learning_curve(estimator, title, X_all, y_all, ylim=(0.7, 1.01), cv=cv, n_jobs=4)

# Conclusion

By analysing the *cross valitation time* and *accuracy score* trade-off, the model that generalizes faster and better to classify the wine quality is **DecisionTreeClassifier** trained with the full training set considering all outliers. Although the outliers removal improves the classification, a better investigation should be performed to verify whether it is a normal behavior or some error. The **accuracy score** is about **92%** and the normalization types used in the training set didn't improve the score significantly. The **learning curve** also shows that our model doesn't suffer from Overfitting and Underfitting, ie, the model has low bias and low variance between training score and cross-validation score ([reference](http://scott.fortmann-roe.com/docs/BiasVariance.html)).